In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import sys
import requests
import re #정규식 사용을 위함 

In [2]:
date="2017.03.28"
url_part1 ="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
url = url_part1+ date
print(url)
res = requests.get(url) 
soup = bs(res.text, "html.parser")

http://dart.fss.or.kr/dsac001/mainAll.do?selectDate=2017.03.28


In [3]:
totNum=soup.find_all("p")[8].text
totNum

'\r\n\t\t[1/9] [총 894건]\r\n\t'

In [4]:
res = requests.get(url) #urllib.request.urlopen이랑 같은거 : 사이트마다 안될시 이용해볼것!!
soup = bs(res.text, "html.parser")

In [5]:
for i in range(0, len(soup.find_all("p"))) :
    print(i)
    print(soup.find_all("p")[i])
    print("--------------")

0
<p id="accessibility"><a href="#layoutMain">본문 바로가기</a></p>
--------------
1
<p class="utill">
<!-- <ul class="topLink"> -->
<a href="/dsag002/loginForm.do"><img alt="로그인" src="/images/common/utill_login.gif" title="로그인"/></a>
<a href="/dsag003/main.do"><img alt="마이페이지" src="/images/common/utill_mypage.gif" title="마이페이지"/></a>
<a href="/dsaa003/searchGuide.do"><img alt="공시업무" src="/images/common/utill_gongsi.gif" title="공시업무"/></a>
<a href="/introduction/content1.do"><img alt="다트소개" src="/images/common/utill_intro.gif" title="다트소개"/></a>
<a href="https://opendart.fss.or.kr/intro/main.do" target="_blank"><img alt="오픈API" src="/images/common/utill_openapi.gif" title="오픈API"/></a>
<a href="/introduction/content6.do"><img alt="RSS이용안내" src="/images/common/utill_rss.gif" title="RSS이용안내"/></a>
<a class="end" href="/sitemap.do"><img alt="사이트맵" src="/images/common/utill_sitemap.gif" title="사이트맵"/></a>
</p>
--------------
2
<p class="tit"><img alt="최근공시" src="/images/common/lnb_recent_disclos

In [6]:
totNum=soup.find_all("p")[8].text

In [7]:
#값 변경 : re.sub(찾을패턴, 찾고나서 변경할 값, 어디서할지 대상 )
# [ : 특수문자라 \표시

re.sub(r"\[총","",totNum)

'\r\n\t\t[1/9]  894건]\r\n\t'

In [8]:
#.* : 그 안에 뭐가 오든 상관 없다. 
# 1.다이렉트 점근 (r없이) : 특수문자 앞에 -\\두개!!
totNum=re.sub("\\[.*\\] \\[", "", totNum)
totNum

'\r\n\t\t총 894건]\r\n\t'

In [9]:
# 2. 간접 접근 
totNum=re.sub(r"총","",totNum)
totNum

'\r\n\t\t 894건]\r\n\t'

In [10]:
totNum=re.sub(r"건\]","",totNum)
totNum

'\r\n\t\t 894\r\n\t'

In [11]:
totNum.strip()

'894'

In [12]:
#strip() : 중간의 공백은 살아있다.!!
a ="      aaaa      bbbbb   ccccc"
a.strip()
#a.replace(찾을패턴, 바꿀값)
a.replace(" ","")

'aaaabbbbbccccc'

In [13]:
totNum=soup.find_all("p")[8].text
totNum

'\r\n\t\t[1/9] [총 894건]\r\n\t'

In [14]:
#처리해야 할 건수에 대한 정보 확인 
totNum = re.sub(r"\[.*\] \[총", "", totNum)
totNum=re.sub(r"건\]","",totNum)
totNum=int(totNum)
totNum


894

In [15]:
#totNum의 값을 가지고 몇 개의 페이지를 롤링해야 하는지?
totPage = (totNum//100) +1
totPage


9

In [16]:
#0~100 =>1~100으로 접근해야해!!!
len(soup.find_all("tr"))

101

In [17]:
soup.find_all("tr")[2]

<tr>
<td class="cen_txt">
						18:46
					</td>
<td>
<span class="nobr1" style="max-width:150px;">
<img alt="코스닥시장" src="/images/ico_kosdaq.gif" title="코스닥시장"/> 
							<a href="/dsae001/selectPopup.ax?selectKey=00364403" onclick="openCorpInfo('00364403'); return false;" title="쏠리드 기업개황 새창">
								쏠리드
							</a>
</span>
</td>
<td>
<a href="/dsaf001/main.do?rcpNo=20170328901048" id="r_20170328901048" onclick="openReportViewer('20170328901048'); return false;" title="불성실공시법인지정예고(공시불이행) 공시뷰어 새창">불성실공시법인지정예고
							
  							(공시불이행)
						</a>
</td>
<td title="코스닥시장본부"><div class="nobr" style="width:95px">코스닥시장본부</div></td>
<td class="cen_txt">2017.03.28</td>
<td class="cen_txt end"> <img alt="본 공시사항은 한국거래소 코스닥시장본부 소관임" hspace="1" src="/images/sub/remark05.gif" title="본 공시사항은 한국거래소 코스닥시장본부 소관임"/></td>
</tr>

In [18]:
#시장종류
Cat= soup.find_all("tr")[2].find_all("td")[1].find("img").get("title") #.get : 태그안의 속성의 값 가져올때
#회사이름
comName = soup.find_all("tr")[2].find_all("a")[0].text.strip()
#회사 ID
testStr = soup.find_all("tr")[2].find_all("a")[0].get("href")
    #방법 :1. re.sub 
    #방법 :2. re.findall
coID =  re.findall(r"[0-9]{8}",testStr)[0] #연속된 숫자 8자리 --> 정규식 : [0-9]{8}
#공시내용/제목
tempStr = soup.find_all("tr")[2].find_all("a")[1].text.strip()
tempStr=tempStr.replace("\r", "")
tempStr=tempStr.replace("\n", "")
tempStr=tempStr.replace("\t", "")
tempStr=tempStr.replace(" ", "")

Content = tempStr
#해당공시 고유값
rcpNo = re.findall(r"[0-9]{14}", soup.find_all("tr")[2].find_all("a")[1].get("href"))[0]
#공시요청날짜
pubDate = soup.find_all("tr")[2].find_all("td", class_="cen_txt")[1].text.strip() # =date
#공시 시간 

pubTime = soup.find_all("tr")[2].find_all("td", class_="cen_txt")[0].text.strip()

In [19]:
#OR로 한번에 지우기 

re.sub(r"\r|\n|\t| ", "", tempStr)

'불성실공시법인지정예고(공시불이행)'

In [20]:
pubDate = soup.find_all("tr")[2].find_all("td", class_="cen_txt")[1].text.strip()
pubDate

'2017.03.28'

In [21]:
 soup.find_all("tr")[2].find_all("td", class_="cen_txt")

[<td class="cen_txt">
 						18:46
 					</td>,
 <td class="cen_txt">2017.03.28</td>,
 <td class="cen_txt end"> <img alt="본 공시사항은 한국거래소 코스닥시장본부 소관임" hspace="1" src="/images/sub/remark05.gif" title="본 공시사항은 한국거래소 코스닥시장본부 소관임"/></td>]

In [3]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import requests
import re 

dartIDDF =pd.DataFrame()
dart_temp =pd.DataFrame()

dart_temp = dart_temp.append(
    {
       "pubTime" :"",
        "cat":"",
        "coID":"",
        "content":"",
        "pubDate":"",
        "pubReqDate" :"",
        "rcpNo":"",
        "comName":""},
    ignore_index =True)

temp_pubTime_idx =dart_temp.columns.get_loc("pubTime")
temp_cat_idx =dart_temp.columns.get_loc("cat")
temp_coID_idx =dart_temp.columns.get_loc("coID")
temp_content_idx =dart_temp.columns.get_loc("content")
temp_pubDate_idx =dart_temp.columns.get_loc("pubDate")
temp_pubReqDate_idx =dart_temp.columns.get_loc("pubReqDate")
temp_rcpNo_idx =dart_temp.columns.get_loc("rcpNo")
temp_comName_idx =dart_temp.columns.get_loc("comName")

#2017.07.24~27일까지의 dart 데이터 추출 
for f in range(24, 27) :
    date="2017.07."+ str(f)
    url_part1 ="""http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="""
    urlDate = url_part1 + date
    res = requests.get(urlDate) 
    soup = bs(res.text, "html.parser")  
    totNum=soup.find_all("p")[8].text
    totNum = re.sub(r"\[.*\] \[총", "", totNum)
    totNum=re.sub(r"건\]","",totNum)
    totNum=int(totNum)
    totPage = (totNum//100) +1
    
    for i in range(1, totPage+1) :
        url_part2 = """&series=&mdayCnt=0&currentPage="""+str(i)
        url = url_part1+ date + url_part2   
        res = requests.get(url) 
        soup = bs(res.text, "html.parser")
        
        tempNumCont = len(soup.find_all("tr"))

        for k in range(1,tempNumCont):
            tempinfo = soup.find_all("tr")[k]

            dart_temp.iat[0, temp_pubTime_idx]=tempinfo.find_all("td", class_="cen_txt")[0].text.strip()
            dart_temp.iat[0, temp_cat_idx]=tempinfo.find_all("td")[1].find("img").get("title")
            dart_temp.iat[0, temp_coID_idx]=re.findall(r"[0-9]{8}",tempinfo.find_all("a")[0].get("href"))[0] 
            tempStr = tempinfo.find_all("a")[1].text.strip()
            tempStr=tempStr.replace("\r", "")
            tempStr=tempStr.replace("\n", "")
            tempStr=tempStr.replace("\t", "")
            tempStr=tempStr.replace(" ", "")
            dart_temp.iat[0, temp_content_idx]=tempStr
            dart_temp.iat[0, temp_pubReqDate_idx]=tempinfo.find_all("td", class_="cen_txt")[1].text.strip()
            if(len(tempinfo.find_all("a"))==2) :
                dart_temp.iat[0, temp_rcpNo_idx]=re.findall(r"[0-9]{14}", tempinfo.find_all("a")[1].get("href"))[0]
            else :
                dart_temp.iat[0, temp_rcpNo_idx]=re.findall(r"[0-9]{14}", tempinfo.find_all("a")[2].get("href"))[0]
            dart_temp.iat[0, temp_comName_idx]=tempinfo.find_all("a")[0].text.strip()
            dart_temp.iat[0, temp_pubDate_idx]=soup.find_all("input")[13].get("value")
            
            dartIDDF=dartIDDF.append(dart_temp,ignore_index = True)

            for j in range(0, len(dart_temp.columns)) :
                dart_temp.iat[0,j] =""

dartIDDF

,cat,coID,comName,content,pubDate,pubReqDate,pubTime,rcpNo
0,코스닥시장,00111865,미래SCI,최대주주변경,2017.07.24,2017.07.24,19:07,20170724900550
1,유가증권시장,00134723,페이퍼코리아,기타경영사항(자율공시),2017.07.24,2017.07.24,18:53,20170724800549
2,유가증권시장,00134723,페이퍼코리아,기타경영사항(자율공시),2017.07.24,2017.07.24,18:49,20170724800548
3,코스닥시장,00268251,다산네트웍스,,2017.07.24,2017.07.24,18:03,20170724900529
4,코스닥시장,00861720,이즈미디어,증권발행실적보고서,2017.07.24,2017.07.24,17:59,20170724000355
...,...,...,...,...,...,...,...,...
894,유가증권시장,00131832,SK디스커버리,투자설명서,2017.07.26,2017.07.26,08:10,20170726000002
895,유가증권시장,00149804,대유에이텍,최대주주등소유주식변동신고서,2017.07.26,2017.07.26,07:58,20170726800003
896,코스닥시장,00308559,코디,주주총회소집공고,2017.07.26,2017.07.26,07:08,20170726000001
897,코스닥시장,00308559,코디,[기재정정]주주총회소집결의,2017.07.26,2017.07.26,07:06,20170726900001


In [23]:
re.findall(r"[0-9]{14}",  soup.find_all("tr")[34].find_all("a")[2].get("href"))

IndexError: list index out of range

In [ ]:
len(soup.find_all("tr")[34].find_all("a"))

In [ ]:
 soup.find_all("tr")[34].find_all("a")

In [26]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

resultDF=pd.DataFrame()
tempDF=pd.DataFrame()

tempDF=tempDF.append({"Cat":"","comName":"","coID":"","Content":"","rcpNo":"","pubTime":"","pubDate":"","reqDate":""},ignore_index=True)

Cat_idx = tempDF.columns.get_loc("Cat")
comName_idx = tempDF.columns.get_loc("comName")
coID_idx = tempDF.columns.get_loc("coID")
Content_idx = tempDF.columns.get_loc("Content")
rcpNo_idx = tempDF.columns.get_loc("rcpNo")
pubTime_idx = tempDF.columns.get_loc("pubTime")
pubDate_idx = tempDF.columns.get_loc("pubDate")
reqDate_idx = tempDF.columns.get_loc("reqDate")

date="2017.03.28"
url_part1="http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="
url_part2="&sort=&series=&mdayCnt=0&currentPage="

url = url_part1 + date + url_part2

res = requests.get(url) 
_soup=bs(res.text, 'html.parser')

totNum=_soup.find_all("p")[8].text
totNum=re.sub(r"\[.*\] \[총","",totNum)
totNum=re.sub(r"건\]","",totNum)
totNum = int(totNum)
totNum
totPage = (totNum//100)+1

for k in range(1,totPage+1):
    date="2017.03.28"
    url_part1="http://dart.fss.or.kr/dsac001/mainAll.do?selectDate="
    url_part2="&sort=&series=&mdayCnt=0&currentPage="

    url = url_part1 + date + url_part2 +str(k)

    res = requests.get(url) 
    soup=bs(res.text, 'html.parser')


    for i in range(1, len(soup.find_all("tr"))):
        tempDF.iat[0,Cat_idx]=soup.find_all("tr")[i].find_all("td")[1].find("img").get("title")
        tempDF.iat[0,comName_idx]=soup.find_all("tr")[i].find("a").text.strip()

        testStr=soup.find_all("tr")[i].find("a").get("href")
        tempDF.iat[0,coID_idx]=re.findall(r"[0-9]{8}",testStr)[0]

        if len(soup.find_all("tr")[i].find_all("a"))==3:
            test=soup.find_all("tr")[i].find_all("a")[2].text.strip()
            test=test.replace("\r","")
            test=test.replace("\n","")
            test=test.replace("\t","")
            test=test.replace(" ","")
            tempDF.iat[0,Content_idx]=Content=test

            test_rcpNo=soup.find_all("tr")[i].find_all("a")[2].get("href")
            tempDF.iat[0,rcpNo_idx]=re.findall(r"[0-9]{14}",test_rcpNo)[0]

        else:            
            test=soup.find_all("tr")[i].find_all("a")[1].text.strip()
            test=test.replace("\r","")
            test=test.replace("\n","")
            test=test.replace("\t","")
            test=test.replace(" ","")
            tempDF.iat[0,Content_idx]=Content=test

            test_rcpNo=soup.find_all("tr")[i].find_all("a")[1].get("href")
            tempDF.iat[0,rcpNo_idx]=re.findall(r"[0-9]{14}",test_rcpNo)[0]

        tempDF.iat[0,pubTime_idx]=soup.find_all("tr")[i].find_all("td", class_="cen_txt")[0].text.strip()
        tempDF.iat[0,pubDate_idx]=date
        tempDF.iat[0,reqDate_idx]=soup.find_all("tr")[i].find_all("td")[4].text

        resultDF=resultDF.append(tempDF,ignore_index=True)

        for j in range(0,len(tempDF.columns)):
            tempDF.iat[0,j]=""
            
resultDF

,Cat,Content,coID,comName,pubDate,pubTime,rcpNo,reqDate
0,코넥스시장,감사보고서제출,01109690,도부마스크,2017.03.28,18:58,20170328601049,2017.03.28
1,코스닥시장,불성실공시법인지정예고(공시불이행),00364403,쏠리드,2017.03.28,18:46,20170328901048,2017.03.28
2,코스닥시장,주권매매거래정지(불성실공시법인지정),00308559,코디,2017.03.28,18:35,20170328901037,2017.03.28
3,코스닥시장,불성실공시법인지정(공시불이행),00308559,코디,2017.03.28,18:35,20170328901047,2017.03.28
4,기타법인,주권매매거래정지(불성실공시법인지정),00207223,위너지스,2017.03.28,18:29,20170328901039,2017.03.28
...,...,...,...,...,...,...,...,...
889,기타법인,임원의변동,00178806,바이더웨이,2017.03.28,07:00,20170327000959,2017.03.28
890,기타법인,[기재정정]감사보고서(2016.12),01083000,케이엔에프인터내셔날,2017.03.28,07:00,20170327000967,2017.03.28
891,기타법인,감사보고서(2016.12),01078868,케이피엑스글로벌,2017.03.28,07:00,20170327000958,2017.03.28
892,기타법인,동일인등출자계열회사와의상품ㆍ용역거래,00186513,코리아세븐,2017.03.28,07:00,20170327000962,2017.03.28


In [27]:
#head 와 tail로 데이터가 정확한지 확인하기!!!!

resultDF.tail(10) 

,Cat,Content,coID,comName,pubDate,pubTime,rcpNo,reqDate
884,코스닥시장,[기재정정]주요사항보고서(타법인주식및출자증권양도결정),00441128,아리온,2017.03.28,07:03,20170328000003,2017.03.28
885,코스닥시장,현금ㆍ현물배당결정,01169434,오가닉티코스메틱,2017.03.28,07:02,20170328900002,2017.03.28
886,코스닥시장,주주총회소집결의,01169434,오가닉티코스메틱,2017.03.28,07:01,20170328900021,2017.03.28
887,기타법인,감사보고서(2016.12),01030266,메카니칼토탈솔루션,2017.03.28,07:00,20170327000963,2017.03.28
888,유가증권시장,특수관계인에대한출자,00112332,미래에셋생명,2017.03.28,07:00,20170327000957,2017.03.28
889,기타법인,임원의변동,00178806,바이더웨이,2017.03.28,07:00,20170327000959,2017.03.28
890,기타법인,[기재정정]감사보고서(2016.12),01083000,케이엔에프인터내셔날,2017.03.28,07:00,20170327000967,2017.03.28
891,기타법인,감사보고서(2016.12),01078868,케이피엑스글로벌,2017.03.28,07:00,20170327000958,2017.03.28
892,기타법인,동일인등출자계열회사와의상품ㆍ용역거래,00186513,코리아세븐,2017.03.28,07:00,20170327000962,2017.03.28
893,기타법인,감사보고서(2016.12),00862826,코릴,2017.03.28,07:00,20170327000968,2017.03.28
